In [27]:
import torch
from torch import Tensor
import torch.nn as nn
import numpy as np
from torchvision import io
from torchvision import transforms as T
from PIL import Image
import pickle
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt
import time
from torch.utils.data import DataLoader, Dataset, random_split
import gc
from typing import Tuple
from torch.nn import functional as F
from collections import OrderedDict
import os
import sys


IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    from IPython.display import display, HTML
    display(HTML("<style>.container { width:80% !important; }</style>"))
    
    from git import Repo

    # Initialize the Git repository object
    repo = Repo(".", search_parent_directories=True)

    # Get the root directory of the Git project
    root_dir = repo.git.rev_parse("--show-toplevel")

    from pathlib import Path

    # Set up path for custom importer modules
    # Data Loader
    importer_module = root_dir + '/dataloader/'
    sys.path.insert(0, importer_module)
    from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

    # Loss

    loss_module = root_dir + '/trainer/loss/'
    sys.path.insert(0, loss_module)
    import loss

    # Trainer
    trainer_module = root_dir + '/trainer/'
    sys.path.insert(0, trainer_module)
    from trainer import Ai4MarsTrainer

    # Insert here your local path to the dataset (temporary)
    data_path = data_path = input("Path to Dataset: ") #'/home/leeoos/Desktop/'

else: # IN_COLAB

    from google.colab import drive
    drive.mount('/content/drive')

    # On Colab the path to the module ti fixed once you have
    # corretly set up the project with gitsetup.ipynb

    # Import Loader
    fixed_path_loader = '/content/drive/MyDrive/Github/visiope/dataloader/'
    sys.path.insert(0, fixed_path_loader)
    from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

    # Import Trainer
    fixed_path_trainer = '/content/drive/MyDrive/Github/visiope/trainer/'
    sys.path.insert(0,fixed_path_trainer )
    from trainer import Ai4MarsTrainer

    # Import Loss
    fixed_path_loss = '/content/drive/MyDrive/Github/visiope/trainer/loss/'
    sys.path.insert(0, fixed_path_loss)
    import loss
    !git clone https://github.com/sithu31296/semantic-segmentation
    %pip install -U gdown
    %pip install -e .
    %pip install einops
    import gdown
    from pathlib import Path

    ckpt = Path('./checkpoints/pretrained/segformer')
    ckpt.mkdir(exist_ok=True, parents=True)

    url = 'https://drive.google.com/uc?id=1-OmW3xRD3WAbJTzktPC-VMOF5WMsN8XT'
    output = './checkpoints/pretrained/segformer/segformer.b3.ade.pth'

    gdown.download(url, output, quiet=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
%cd semantic-segmentation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/semantic-segmentation/semantic-segmentation


In [28]:
test = False
if test:
    class Generator(nn.Module):
        def __init__(self, nc):
            self.nc = nc
            super(Generator, self).__init__()


        def forward(self, embeds):
            ngf = 64
            nz = [[None] *6]*4

            j = 0
            for embed in embeds:
                nz[j][0] = embed.size()[1]
                for i in range(1,6):
                    if i < 4:
                        nz[j][i] = nz[i-1]//2
                    else:
                        nz[j][i] = nz[i-1]//4
                j = j+1
                print(nz)
            self.main = nn.Sequential(
                #reduction of dimensionality To Be Changed in conv... maybe
                # input is Z, going into a convolution
                nn.ConvTranspose2d( nz[6], ngf * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 8),
                nn.ReLU(True),
                # state size. ``(ngf*8) x 4 x 4``
                nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                # state size. ``(ngf*4) x 8 x 8``
                nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                # state size. ``(ngf*2) x 16 x 16``
                nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                # state size. ``(ngf) x 32 x 32``
                nn.ConvTranspose2d( ngf, self.nc, 4, 2, 1, bias=False),
                nn.Tanh()
                # state size. ``(nc) x 64 x 64``
            )

            return self.main(embed)

In [29]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        ngf = 64
        nc = 1
        print(nz)
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

In [10]:
# Python program to join tensors in PyTorch
# import necessary library
import torch

# create tensors

T1 = torch.zeros((2,2))
T2 = torch.Tensor([[1,2,3,4],[0,3,4,1]])
T3 = torch.Tensor([[4,3,2,5,6,5], [0,3,4,1,6,8]])

print(T1)

# join (concatenate) above tensors using torch.cat()
T = torch.cat((T1,T2,T3), dim=1)
# print final tensor after concatenation
print("T:",T)

tensor([[0., 0.],
        [0., 0.]])
T: tensor([[0., 0., 1., 2., 3., 4., 4., 3., 2., 5., 6., 5.],
        [0., 0., 0., 3., 4., 1., 0., 3., 4., 1., 6., 8.]])


In [30]:
test1 = True
if test1:
    class Generator(nn.Module):
        def __init__(self, nz):
            super(Generator, self).__init__()
            ngf = 64
            nc = 5
            self.main = nn.Sequential()
            self.main = nn.Sequential(
                # input is Z, going into a convolution
                #in_channels, out_channels, kernel_size, stride=1, padding=0,
                nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 8),
                nn.ReLU(True),
                # state size. ``(ngf*8) x 4 x 4``
                nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                # state size. ``(ngf*4) x 8 x 8``
                nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                # state size. ``(ngf*2) x 16 x 16``
                nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                # state size. ``(ngf) x 32 x 32``
                nn.ConvTranspose2d( ngf, ngf//2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf//2),
                nn.ReLU(True),
                # state size. ``(ngf/2) x 64 x 64``
                nn.ConvTranspose2d( ngf//2, nc, 4, 2, 1, bias=False),
                nn.Tanh()
                # state size. ``(ngc) x 128 x 128``
                # state size. ``(ngf/4) x 128 x 128``
                #nn.BatchNorm2d(ngf),
                #nn.ReLU(True),
                #nn.ConvTranspose2d( ngf/4, nc, 4, 2, 1, bias=False),
                #nn.Tanh()
                # state size. ``(nc) x 256 x 256``
            )

        def forward(self, input):
            return self.main(input)

In [31]:
class MLP(nn.Module):
    def __init__(self, dim, embed_dim):
        super().__init__()
        self.proj = nn.Linear(dim, embed_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x


class ConvModule(nn.Module):
    def __init__(self, c1, c2):
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, 1, bias=False)
        self.bn = nn.BatchNorm2d(c2)        # use SyncBN in original
        self.activate = nn.ReLU(True)

    def forward(self, x: Tensor) -> Tensor:
        return self.activate(self.bn(self.conv(x)))


class SegFormerHead(nn.Module):
    def __init__(self, dims: list, embed_dim: int = 256, num_classes: int = 19):
        super().__init__()
        for i, dim in enumerate(dims):
            self.add_module(f"linear_c{i+1}", MLP(dim, embed_dim))

        self.linear_fuse = ConvModule(embed_dim*4, embed_dim)
        self.linear_pred = nn.Conv2d(embed_dim, num_classes, 1)
        self.dropout = nn.Dropout2d(0.1)

    def forward(self, features: Tuple[Tensor, Tensor, Tensor, Tensor]) -> Tensor:
        B, _, H, W = features[0].shape
        outs = [self.linear_c1(features[0]).permute(0, 2, 1).reshape(B, -1, *features[0].shape[-2:])]

        for i, feature in enumerate(features[1:]):
            cf = eval(f"self.linear_c{i+2}")(feature).permute(0, 2, 1).reshape(B, -1, *feature.shape[-2:])
            outs.append(F.interpolate(cf, size=(H, W), mode='bilinear', align_corners=False))

        seg = self.linear_fuse(torch.cat(outs[::-1], dim=1))
        seg = self.linear_pred(self.dropout(seg))
        return seg

In [32]:
class MLP(nn.Module):
    def __init__(self, dim, embed_dim):
        super().__init__()
        self.proj = nn.Linear(dim, embed_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        batch_size = x.shape[0]
        return x.reshape(batch_size, 1, -1)

class FlattenFinal(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        batch_size = x.shape[0]
        return x.reshape(batch_size, -1, 1, 1)

class Conv2DModule(nn.Module):
    def __init__(self, cin, cout, ks, s):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, ks, stride=s, bias=False)
        self.bn = nn.BatchNorm2d(cout)        # use SyncBN in original
        self.activate = nn.ReLU(True)

    def forward(self, x: Tensor) -> Tensor:
        return self.activate(self.bn(self.conv(x)))

class Conv1DModule(nn.Module):
    def __init__(self, cin, cout, ks, s):
        super().__init__()
        self.conv = nn.Conv1d(cin, cout, ks, stride=s, bias=False)
        self.bn = nn.BatchNorm1d(cout)        # use SyncBN in original
        self.activate = nn.ReLU(True)

    def forward(self, x: Tensor) -> Tensor:
        return self.activate(self.bn(self.conv(x)))

class downsample(nn.Module):
    def __init__(self, C, H, W):
        super().__init__()
        self.downsample = nn.Sequential()
        i = 0
        alt = True
        if H*W > 128:
            while(C>4):

                if H*W*C < 500:
                    break

                if H//2<3 or W//2<3:
                    if alt:
                        self.downsample.append(Flatten())
                        C = C*H*W
                        alt = False
                    self.downsample.append(Conv1DModule(1, 1, 3, 1))
                    C = C//2
                    i = i+1
                elif H//2<9 or W//2<9:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 1))
                    C = C//2
                    i = i+1
                    H = H-2
                    W = W-2
                else:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 2))
                    C = C//2
                    i = i+1
                    H = H//2 -1
                    W = W//2 -1

        else:
            while(C>16):

                if H*W*C < 500:
                    break

                if H//2<3 or W//2<3:
                    if alt:
                        self.downsample.append(Flatten())
                        C = C*H*W
                        alt = False
                    self.downsample.append(Conv1DModule(1, 1, 3, 1))
                    C = C//2
                    i = i+1
                elif H//2<9 or W//2<9:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 1))
                    C = C//2
                    i = i+1
                    H = H-2
                    W = W-2
                else:
                    self.downsample.append(Conv2DModule(C, C//2, 3, 2))
                    C = C//2
                    i = i+1
                    H = H//2 -1
                    W = W//2 -1
        self.downsample.append(FlattenFinal())

    def forward(self, x: Tensor) -> Tensor:
        return self.downsample(x)



In [33]:
class SegFormerHeadGen(nn.Module):

    def __init__(self, num_img):
      self.num_img = num_img
      super().__init__()


    def forward(self, features: Tuple[Tensor, Tensor, Tensor, Tensor]) -> Tensor:
      x = features[self.num_img]
      downsampler = downsample(x.shape[1],x.shape[2],x.shape[3]).to(device)


      downsampler(x).size()
      self.G = Generator(downsampler(x).size()[1]).to(device)
      image = self.G(downsampler(x))


      return image

In [34]:
from semseg.models.base import BaseModel
from semseg.models.heads import SegFormerHead
#from resDecode import ResDecode


class SegFormerpp(BaseModel):
    def __init__(self, backbone: str = 'MiT-B0', num_classes: int = 19, head: str = 'B0', num_img: int = -1) -> None:
        super().__init__(backbone, num_classes)
        self.num_classes = num_classes
        self.head = head
        self.decode = SegFormerHead(self.backbone.channels, 256 if 'B0' in backbone or 'B1' in backbone else 768, 3)
        self.newDecode = SegFormerHeadGen(num_img)
        self.newDecode.to(device)
        self.apply(self._init_weights)


    def forward(self, x: Tensor) -> Tensor:
        y = self.backbone(x)
        #y = self.decode(y)
        #y = F.interpolate(y, size=x.shape[2:], mode='bilinear', align_corners=False)
        #embeds = []
        y = self.newDecode(y)
        '''
        y = self.decode(y)
        self.upsample = nn.Sequential(
        torch.nn.ConvTranspose2d(y.shape[1],self.num_classes//2,3,stride=2,output_padding=1, padding=1),
        torch.nn.ConvTranspose2d(self.num_classes//2,self.num_classes,3,stride=2,output_padding=1, padding=1),
                                )
        y = self.upsample(y).to(device)
        '''
        return y


In [47]:
from semseg.models import *
#from segFormerpp import SegFormerpp

model = eval('SegFormerpp')(
    backbone='MiT-B4',
    num_classes=5,
    num_img = 0
)

try:
    model.load_state_dict(torch.load('checkpointcheckpoints/pretrained/segformer/segformer.b3.ade.pth'))
except:
    print("Download a pretrained model's weights from the result table.")
model.eval()
print('Loaded Model')

Download a pretrained model's weights from the result table.
Loaded Model


In [36]:
if not IN_COLAB:
  %cd ../../dataloader/
  from load import Ai4MarsData
  %cd ../models/semantic-segmentation

In [37]:
if IN_COLAB  :
    #!git clone https://github.com/airoprojects/visiope
    # %cd visiope/dataloader/
    # from load import Ai4MarsData
    # %cd ../..
    # with open('/content/drive/MyDrive/Dataset/data_loader.pkl', 'rb') as f:
    #     data_loader = pickle.load(f)


    # data_loader['dataloader'].dataset.setPermuteX((0,3,1,2))
    # data_loader['dataloader'].dataset.setDevice(device,0)
    # data_loader['dataloader'].dataset.setDevice(device,1)
    # #data_loader['dataloader'].dataset.setdevice(device)

    # train_loader, test_loader, validation_loader = data_loader['dataloader'].dataset.splitLoader(80,1)


    if not(os.path.exists('/content/dataset/X_200') and
        os.path.exists('/content/dataset/y_200')):

        import gdown

        # # get url of np dataset (temporarerly my drive)
        # url_X = 'https://drive.google.com/uc?id=1HLAQgjZbGa3lMzdyIvgykCiUJ6P4OaEX'
        # url_y = 'https://drive.google.com/uc?id=1Ue74LEe0WlEnFxRcIl19WyvXR1EyYIsS'

        # # download np dataset on runtime env
        # data_path = '/content/dataset/'
        # gdown.download(url_X, data_path, quiet=False)
        # gdown.download(url_y, data_path, quiet=False)

        # get url of np dataset (temporarerly my drive)
        url_X_200 = 'https://drive.google.com/uc?id=1HLAQgjZbGa3lMzdyIvgykCiUJ6P4OaEX'
        url_y_200 = 'https://drive.google.com/uc?id=1Ue74LEe0WlEnFxRcIl19WyvXR1EyYIsS'

        url_X_1000 = 'https://drive.google.com/uc?id=1zvXKK1qc2PNbAMyq0XWfqrhAWB5kO3yO'
        url_y_1000 = 'https://drive.google.com/uc?id=1gmnWAGjZJYt-VIpWRK_cEOXQfFfuFAwe'

        # download np dataset on runtime env
        data_path = '/content/dataset/'
        gdown.download(url_X_200, data_path, quiet=False)
        gdown.download(url_y_200, data_path, quiet=False)


    X = np.load('/content/dataset/X_200.npy')
    y = np.load('/content/dataset/y_200.npy')

    processor = Ai4MarsProcessor()
    train_set, test_set, val_set = processor(X, y, [0.60, 0.20, 0.20])

    train_set.resize(128)
    test_set.resize(128)
    val_set.resize(128)

    train_set.conversion('f')
    test_set.conversion('f')
    val_set.conversion('f')

    train_set.set_grad()
    test_set.set_grad()
    val_set.set_grad()

    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=16, shuffle=True)

Cloning into 'visiope'...
remote: Enumerating objects: 954, done.
remote: Counting objects: 100% (358/358), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 954 (delta 197), reused 262 (delta 140), pack-reused 596
Receiving objects: 100% (954/954), 71.42 MiB | 13.95 MiB/s, done.
Resolving deltas: 100% (510/510), done.


Downloading...
From (uriginal): https://drive.google.com/uc?id=1HLAQgjZbGa3lMzdyIvgykCiUJ6P4OaEX
From (redirected): https://drive.google.com/uc?id=1HLAQgjZbGa3lMzdyIvgykCiUJ6P4OaEX&confirm=t&uuid=07c0d5b9-ab3e-4868-add1-9d73cd904df6
To: /content/dataset/X_200.npy
100%|██████████| 629M/629M [00:05<00:00, 117MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Ue74LEe0WlEnFxRcIl19WyvXR1EyYIsS
From (redirected): https://drive.google.com/uc?id=1Ue74LEe0WlEnFxRcIl19WyvXR1EyYIsS&confirm=t&uuid=4477ef1a-5a6b-4628-9039-cae00c54af6c
To: /content/dataset/y_200.npy
100%|██████████| 210M/210M [00:01<00:00, 107MB/s]


In [38]:
#local
if not IN_COLAB:

    with open('../../dataloader/data_loader.pkl', 'rb') as f:
        data_loader = pickle.load(f)

    data_loader['dataloader'].dataset.setPermuteX((0,3,1,2))
    data_loader['dataloader'].dataset.resize(128)
    #data_loader['dataloader'].dataset.convertion(0)
    data_loader['dataloader'].dataset.setDevice(device,0)
    data_loader['dataloader'].dataset.setDevice(device,1)
    #data_loader['dataloader'].dataset.setdevice(device)

    train_loader, test_loader, validation_loader = data_loader['dataloader'].dataset.splitLoader(80,1)


In [39]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

def show_image(imgs):
    imgs = imgs.permute(2,0,1)
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

'''
  print(img.device)
  print(img.shape)
  if (img.device != 'cpu'):
    img.to('cpu')
    print('test')
  print(img.device)
  plt.imshow(img)
  plt.show()
'''

"\n  print(img.device)\n  print(img.shape)\n  if (img.device != 'cpu'):\n    img.to('cpu')\n    print('test')\n  print(img.device)\n  plt.imshow(img)\n  plt.show()\n"

In [40]:
#local
if not IN_COLAB:
  %cd ../../loss/
  from trainer_module import trainer
  %cd ../models/semantic-segmentation

In [41]:
print(len(test_loader.dataset))

40


In [42]:
'''
if IN_COLAB  :
    %cd visiope/loss/
    from trainer_module import trainer
    %cd ../..
'''

'\nif IN_COLAB  :\n    %cd visiope/loss/\n    from trainer_module import trainer\n    %cd ../..\n'

In [43]:
""" Trainer module for MER-Segmentation """

from datetime import datetime
import torch

# Main trainer function
def trainer(parameters, multiple_epochs=False, epoch_index=0, tb_writer=0):
    model = parameters['model']
    device = parameters['device']
    model.to(device)
    if multiple_epochs:
        train_multiple_epoch(parameters,  model, EPOCHS=100)

    else:
        train_one_epoch(parameters, epoch_index, tb_writer, model)

# IMPORTANT: FIND OUT ABOUT TB_WRITER
def train_one_epoch(parameters, epoch_index, tb_writer, model):

    # Initialization of training parameters

    loss_fn = parameters['loss']
    optimizer = parameters['optimizer']
    training_set = parameters['training']
    device = parameters['device']

    # To keep track of the last loss when the function is executed through multiple epochs
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_set):
        # Every data instance is an input + label pair
        inputs, labels = data
        #inputs = inputs.permute(0,3,1,2).to(device)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        labels = labels.long()

        labels = labels.squeeze()

        inputs = inputs.to(device)
        labels = labels.to(device)
        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        #print(labels.type())



        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        running_loss += loss.item()

        # IMPORTANT: UNCOMMENT THIS PART AFTER FINDING OUT ABOUT TB_WRITER
        '''
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_set) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
        '''
        inputs.detach()
        labels.detach()
        del inputs
        del labels

    last_loss = running_loss/(i+1)

    return last_loss


def train_multiple_epoch(parameters, model, EPOCHS=100):

    # Initialization of training parameters
    loss_fn = parameters['loss']
    #optimizer = parameters['optimizer']
    #training_set = parameters['training']
    validation_set = parameters['validation']

    # Initialization of report parameters
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = 0 #SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    epoch_number = 0 # just a counter
    best_vloss = 1_000_000.

    for epoch in range(EPOCHS):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)

        start = time.time()

        avg_loss = train_one_epoch(parameters, epoch_number, writer, model)

        end = time.time()
        print("epoch time: "+str(end-start)+"s")

        # We don't need gradients on to do reporting
        model.train(False)

        # Validation loss
        running_vloss = 0.0
        for i, vdata in enumerate(validation_set):
            vinputs, vlabels = vdata

            vlabels = vlabels.squeeze()
            vlabels = vlabels.long()

            vinputs = vinputs.to(device)
            vlabels = vlabels.to(device)

            voutputs = model(vinputs)

            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # IMPORTANT: UNCOMMENT THIS PART AFTER FINDING OUT ABOUT TB_WRITER
        '''
        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                        { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch_number + 1)
        writer.flush()
        '''

        '''
        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = 'model_{}_{}'.format(timestamp, epoch_number)
            torch.save(model.state_dict(), model_path)
        '''
        #torch.cuda.memory_summary()
        vinputs.detach()
        vlabels.detach()
        del vinputs
        del vlabels

        torch.cuda.empty_cache()
        gc.collect()

        #torch.cuda.memory_summary()

        epoch_number += 1

if __name__ == '__main__':

    # This variable needs to be initialized with dummy values/data to test loss integration
    '''
    model = 0
    optimizer = 0
    loss_fn = 0
    training_set = 0
    validation_set = 0

    parameters = {
                    'model' : model,
                    'loss' : loss_fn,
                    'optimizer' : optimizer,
                    'training' : training_set,
                    'validation': validation_set
    }
    '''


In [ ]:
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss().to(device)
training_set = train_loader
validation_set = test_loader

parameters = {
    'model' : model,
    'loss' : loss_fn,
    'optimizer' : optimizer,
    'training' : training_set,
    'validation' : validation_set,
    'device' : device
}
#trainer(parameters)
trainer(parameters,multiple_epochs=True)

EPOCH 1:
epoch time: 1.701798677444458s
LOSS train 1.675107091665268 valid 1.6850411891937256
EPOCH 2:
epoch time: 1.5604867935180664s
LOSS train 1.6485661268234253 valid 1.6498510837554932
EPOCH 3:
epoch time: 1.3964083194732666s
LOSS train 1.6778640151023865 valid 1.710861086845398
EPOCH 4:
epoch time: 1.3540217876434326s
LOSS train 1.6554960310459137 valid 1.6965243816375732
EPOCH 5:
epoch time: 1.3792519569396973s
LOSS train 1.6761048436164856 valid 1.6738336086273193
EPOCH 6:
epoch time: 1.3901605606079102s
LOSS train 1.6646405160427094 valid 1.6377824544906616
EPOCH 7:
epoch time: 1.4117755889892578s
LOSS train 1.6803098022937775 valid 1.6948928833007812
EPOCH 8:
epoch time: 1.6654863357543945s
LOSS train 1.647966355085373 valid 1.6606907844543457
EPOCH 9:
epoch time: 1.5083017349243164s
LOSS train 1.7042682766914368 valid 1.6818288564682007
EPOCH 10:
epoch time: 1.420896291732788s
LOSS train 1.654944896697998 valid 1.6639127731323242
EPOCH 11:
epoch time: 1.4395742416381836s
LOS

In [45]:
from semseg.datasets import *

#model = model.to(device)
predictions = []
start = time.time()
#print('test')
# Use torch.no_grad() to disable gradient computation during testing
with torch.no_grad():

    for images, labels in train_loader:
        # Move the data to the desired device
        #print(images.shape)
        #images = images.permute(0,3,1,2).to(device)
        #images = images.permute(2,0,1).to(device)
        #images = images [None, :, :, :]
        #print(images.shape)

        #print(images.shape)
        #labels = labels.to(device)

        # Forward pass to get the predictions
        with torch.inference_mode():
          prediction = model(images)
        #print(prediction)
        prediction = prediction.softmax(1).argmax(1).to(int)
        #prediction = prediction.round().to(int)
        #print(prediction.shape)
        un = prediction.unique()
        #print(un)
        palette = eval('ADE20K').PALETTE.to(device)
        prediction_map = palette[prediction].squeeze().to(torch.uint8)
        #print(type(prediction_map))
        show_image(prediction_map)
        predictions.append(prediction_map)

end = time.time()
print(end-start)

RuntimeError: ignored

In [ ]:
torch.randn(64, 512, 1, 1, device=device).size()